### Задача 1
---
На первом складе (А1) содержится сталь двух марок: 3200 т марки «а» и 5400 т марки «б». Сталь этих марок содержится также и на втором складе (А2) в следующих количествах: 2700 т марки «а» и 5100 т марки «б». Сталь должна быть вывезена в два пункта потребления: в пункт В1 необходимо поставить 1500 т стали марки «а», 6300 т марки «б» и остальные 900 т стали любой марки. Аналогично второй пункт потребления В2 должен получить 5000 т стали, из них 1700 т марки «а» и 3000 т марки «б». Известно, что 1 т стали марки «а» может заменить 0.7 т стали марки «б» (то есть, вместо стали марки «б» можно использовать соответствующее количество стали марки «а», но не наоборот).
Стоимость перевозок в денежных единицах (ДЕ) за тонну составляют: из пункта А1 в пункты В1 и В2 3.4 ДЕ и 4.3 ДЕ, и из пункта А2 в В1 и В2, соответственно, 4.8 ДЕ и 4.3 ДЕ. Требуется найти план перевозок стали, минимизирующий затраты на перевозки; полученное решение необходимо исследовать.
1. Провести параметрическое изменение правых частей:
     - если в решении отсутствуют перевозки с заменой марок стали («а» на «б»), то провести постепенное изменение запасов на обоих складах (увеличение - для стали марки «а», и уменьшение для стали марки «б») до тех пор, пока по крайней мере 100 т стали марки «а» не будут направлены на замену стали марки «б»;
     - если в решении присутствуют такие перевозки, то провести обратное постепенное изменение запасов, пока перевозки с заменой марок стали не исчезнут (рассмотреть вариант только с увеличением запасов стали марки «б» без изменения запасов стали марки «а» на обоих складах).
2. Провести параметрическое изменение целевой функции:
    - предварительно существено увеличить запасы стали обеих марок (в правых частях), чтобы на предпочтительность перевозок стали той или иной марки не влияли ограничения по запасам;
    - убедиться в отсутствии замен (или в их наличии), для чего заново решить задачу;
    - затем провести постепенное изменение (увеличение или уменьшение - по смыслу) стоимости перевозок стали марки «б» (при прежней стоимости перевозок стали марки «а») до появления хотя бы 100 т замены (или до исчезновения замен).


In [1]:
from cvxopt import matrix, printing, solvers

printing.options['dformat'] = '%7.5f'

A1_to_B1 = 3.4
A1_to_B2 = 4.3
A2_to_B1 = 4.8
A2_to_B2 = 4.3

A1_a = 3200
A1_b = 5400
A2_a = 2700
A2_b = 5100

B1_a   = 1500.
B1_b   = 6300
B1_any = 900
B2_a   = 1700
B2_b   = 3000
B2_any = 300

B1_overall = B1_a + B1_b + B1_any
B2_overall = B2_a + B2_b + B2_any
B1_w_o_a   = B1_overall - B1_a
B2_w_o_a   = B2_overall - B2_a

a_to_b = .7 # коэффициент замещения (а) на (б)

c = matrix([
    A1_to_B1, # Вывозим из A1 в B1 (а)
    A1_to_B1, # Вывозим из A1 в B1 (б)
    A1_to_B1, # Вывозим из A1 в B1 (а конверченную в б)
    A1_to_B2, # Вывозим из A1 в B2 (а)
    A1_to_B2, # Вывозим из A1 в B2 (б)
    A1_to_B2, # Вывозим из A1 в B2 (а конверченную в б)
    A2_to_B1, # Вывозим из A2 в B1 (а)
    A2_to_B1, # Вывозим из A2 в B1 (б)
    A2_to_B1, # Вывозим из A2 в B1 (а конверченную в б)
    A2_to_B2, # Вывозим из A2 в B2 (а)
    A2_to_B2, # Вывозим из A2 в B2 (б)
    A2_to_B2  # Вывозим из A2 в B2 (а конверченную в б)
])
G = matrix([
    [-1.,0,0,0,0,0,0,0,0,0,0,0],
    [0,-1,0,0,0,0,0,0,0,0,0,0],
    [0,0,-1,0,0,0,0,0,0,0,0,0],
    [0,0,0,-1,0,0,0,0,0,0,0,0],
    [0,0,0,0,-1,0,0,0,0,0,0,0],
    [0,0,0,0,0,-1,0,0,0,0,0,0],
    [0,0,0,0,0,0,-1,0,0,0,0,0],
    [0,0,0,0,0,0,0,-1,0,0,0,0],
    [0,0,0,0,0,0,0,0,-1,0,0,0],
    [0,0,0,0,0,0,0,0,0,-1,0,0],
    [0,0,0,0,0,0,0,0,0,0,-1,0],
    [0,0,0,0,0,0,0,0,0,0,0,-1],
    
    [1.,0,1,1,0,1,0,0,0,0,0,0],
    [0,1,0,0,1,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,1,0,1,1,0,1],
    [0,0,0,0,0,0,0,1,0,0,1,0],
    
    [-1,0,0,0,0,0,-1,0,0,0,0,0],
    [0,0,0,-1,0,0,0,0,0,-1,0,0],
    
    
    [0,-1,-a_to_b,0,0,0,0,-1,-a_to_b,0,0,0],
    [0,0,0,0,-1,-a_to_b,0,0,0,0,-1,-a_to_b],
]).T
h = matrix([
    0., # Все наши вывезенные стали должны быть больше нуля, 
    0,  # но система жрёт только <=, поэтому, например, 
    0,  # x_0 >= 0 нужно переделать в -x_0 <= 0
    0,  # Все коэффициенты для иксов в матрице G,
    0,  # а чиселки справа - в матрице h
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    
    A1_a,  # Вывезти с склада A1 стали (а) нельзя больше, чем в этом складе есть
    A1_b,  # Вывезти с склада A1 стали (б) нельзя больше, чем в этом складе есть
    A2_a,  # Вывезти с склада A2 стали (а) нельзя больше, чем в этом складе есть
    A2_b,  # Вывезти с склада A2 стали (б) нельзя больше, чем в этом складе есть
    
    -B1_a, # а минимум 1500
    -B2_a, # а минимум 1700
    
    -B1_b, # б минимум 6300
    -B2_b, # б минимум 3000
    
])
A = matrix([
    [1,1,a_to_b,0,0,0,1,1,a_to_b,0,0,0], # В B1 вывозим 8700 стали всего
    [0,0,0,1,1,a_to_b,0,0,0,1,1,a_to_b], # В B2 вывозим 5000 стали всего
]).T
b = matrix([8700., 5000])    # Коэффициенты для иксов в равенствах в матрице A, а числа справа - в b

s = solvers.lp(c, G, h, A, b, solver = 'glpk')

print(s['primal objective'])
print(s['x'])

52340.0
[2400.00000]
[5400.00000]
[   0.00000]
[ 800.00000]
[   0.00000]
[   0.00000]
[   0.00000]
[ 900.00000]
[   0.00000]
[1200.00000]
[3000.00000]
[   0.00000]



## Получилось:

- С A1 в B1 2400 (а)
- С A1 в B1 5400 (б)
- С A1 в B1 0    (а_б)
- С A1 в B2 800 (а)
- С A1 в B2 0    (б)
- С A1 в B2 0    (а_б)
- С A2 в B1 0    (а)
- С A2 в B1 900  (б)
- С A2 в B1 0    (а_б)
- С A2 в B2 1200 (а)
- С A2 в B2 3000 (б)
- С A2 в B2 0    (а_б)

Это скок тонн вывезти.

Целевая функция (суммарная стоимость всех перевозок) составила 52340 ДЕ

# TODO Расписать по красоте, мб в коде тоже там как-то оформить в виде датафрейма/да хоть printf с s['x'][0], s['x'][1], и т.д.

Также не забудь, что там в качестве "любая сталь" в обоих случаях использовалась сталь (а).

Как видно, сталь нигде не менялась, поэтому будем делать в первом пункте исследований первый подпункт.  
Уменьшаем содержимое б, увеличиваем а.

Не менялась, потому как б стали достаточно, чтобы покрыть потребность, а замена стали невыгодна (0.7).

In [2]:
ch = 0

while s['status'] != 'optimal' or s['x'][2] + s['x'][5] + s['x'][8] + s['x'][11] < 100:
    ch += 1
    A1_a += ch
    A1_b -= ch
    A2_a += ch
    A2_b -= ch
    h = matrix([
        0., # Все наши вывезенные стали должны быть больше нуля, 
        0,  # но система жрёт только <=, поэтому, например, 
        0,  # x_0 >= 0 нужно переделать в -x_0 <= 0
        0,  # Все коэффициенты для иксов в матрице G,
        0,  # а чиселки справа - в матрице h
        0,
        0,
        0,
        0,
        0,
        0,
        0,

        A1_a,  # Вывезти с склада A1 стали (а) нельзя больше, чем в этом складе есть
        A1_b,  # Вывезти с склада A1 стали (б) нельзя больше, чем в этом складе есть
        A2_a,  # Вывезти с склада A2 стали (а) нельзя больше, чем в этом складе есть
        A2_b,  # Вывезти с склада A2 стали (б) нельзя больше, чем в этом складе есть

        -B1_a, # а минимум 1500 для B1
        -B2_a, # а минимум 1700 для B2

        -B1_b, # б минимум 6300 для B1
        -B2_b, # б минимум 3000 для B2
    ])
    
    s = solvers.lp(c, G, h, A, b, solver = 'glpk')

print(f'A1_a:\t{A1_a}\nA1_b:\t{A1_b}\nA2_a:\t{A2_a}\nA2_b:\t{A2_b}\n')
print(s['x'])
print(f"a_to_b:\t{s['x'][2] + s['x'][5] + s['x'][8] + s['x'][11]}")

A1_a:	3866
A1_b:	4734
A2_a:	3366
A2_b:	4434

[2400.00000]
[4734.00000]
[ 188.57143]
[1277.42857]
[   0.00000]
[   0.00000]
[   0.00000]
[1434.00000]
[   0.00000]
[ 722.57143]
[3000.00000]
[   0.00000]

a_to_b:	188.57142857142856


При вот таких вот запасах в A, началась замена (мы увеличивали по одной тонне запасы сразу в А1 и А2 для а, и уменьшали по одной тонне сразу в А1 и А2 для б).

In [3]:
mult_by = 5

A1_a = 3200 * mult_by
A1_b = 5400 * mult_by
A2_a = 2700 * mult_by
A2_b = 5100 * mult_by

B1_a   = 1500.
B1_b   = 6300
B1_any = 900
B2_a   = 1700
B2_b   = 3000
B2_any = 300

B1_overall = B1_a + B1_b + B1_any
B2_overall = B2_a + B2_b + B2_any
B1_w_o_a   = B1_overall - B1_a
B2_w_o_a   = B2_overall - B2_a

a_to_b = .7 # коэффициент замещения (а) на (б)

c = matrix([
    A1_to_B1, # Вывозим из A1 в B1 (а)
    A1_to_B1, # Вывозим из A1 в B1 (б)
    A1_to_B1, # Вывозим из A1 в B1 (а конверченную в б)
    A1_to_B2, # Вывозим из A1 в B2 (а)
    A1_to_B2, # Вывозим из A1 в B2 (б)
    A1_to_B2, # Вывозим из A1 в B2 (а конверченную в б)
    A2_to_B1, # Вывозим из A2 в B1 (а)
    A2_to_B1, # Вывозим из A2 в B1 (б)
    A2_to_B1, # Вывозим из A2 в B1 (а конверченную в б)
    A2_to_B2, # Вывозим из A2 в B2 (а)
    A2_to_B2, # Вывозим из A2 в B2 (б)
    A2_to_B2  # Вывозим из A2 в B2 (а конверченную в б)
])
G = matrix([
    [-1.,0,0,0,0,0,0,0,0,0,0,0],
    [0,-1,0,0,0,0,0,0,0,0,0,0],
    [0,0,-1,0,0,0,0,0,0,0,0,0],
    [0,0,0,-1,0,0,0,0,0,0,0,0],
    [0,0,0,0,-1,0,0,0,0,0,0,0],
    [0,0,0,0,0,-1,0,0,0,0,0,0],
    [0,0,0,0,0,0,-1,0,0,0,0,0],
    [0,0,0,0,0,0,0,-1,0,0,0,0],
    [0,0,0,0,0,0,0,0,-1,0,0,0],
    [0,0,0,0,0,0,0,0,0,-1,0,0],
    [0,0,0,0,0,0,0,0,0,0,-1,0],
    [0,0,0,0,0,0,0,0,0,0,0,-1],
    
    [1.,0,1,1,0,1,0,0,0,0,0,0],
    [0,1,0,0,1,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,1,0,1,1,0,1],
    [0,0,0,0,0,0,0,1,0,0,1,0],
    
    [-1,0,0,0,0,0,-1,0,0,0,0,0],
    [0,0,0,-1,0,0,0,0,0,-1,0,0],
    
    
    [0,-1,-a_to_b,0,0,0,0,-1,-a_to_b,0,0,0],
    [0,0,0,0,-1,-a_to_b,0,0,0,0,-1,-a_to_b],
]).T
h = matrix([
    0., # Все наши вывезенные стали должны быть больше нуля, 
    0,  # но система жрёт только <=, поэтому, например, 
    0,  # x_0 >= 0 нужно переделать в -x_0 <= 0
    0,  # Все коэффициенты для иксов в матрице G,
    0,  # а чиселки справа - в матрице h
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    
    A1_a,  # Вывезти с склада A1 стали (а) нельзя больше, чем в этом складе есть
    A1_b,  # Вывезти с склада A1 стали (б) нельзя больше, чем в этом складе есть
    A2_a,  # Вывезти с склада A2 стали (а) нельзя больше, чем в этом складе есть
    A2_b,  # Вывезти с склада A2 стали (б) нельзя больше, чем в этом складе есть
    
    -B1_a, # а минимум 1500
    -B2_a, # а минимум 1700
    
    -B1_b, # б минимум 6300
    -B2_b, # б минимум 3000
    
])
A = matrix([
    [1,1,a_to_b,0,0,0,1,1,a_to_b,0,0,0], # В B1 вывозим 8700 стали всего
    [0,0,0,1,1,a_to_b,0,0,0,1,1,a_to_b], # В B2 вывозим 5000 стали всего
]).T
b = matrix([8700., 5000])    # Коэффициенты для иксов в равенствах в матрице A, а числа справа - в b

s = solvers.lp(c, G, h, A, b, solver = 'glpk')

print(s['primal objective'])
print(s['x'])

51080.0
[2400.00000]
[6300.00000]
[   0.00000]
[2000.00000]
[3000.00000]
[   0.00000]
[   0.00000]
[   0.00000]
[   0.00000]
[   0.00000]
[   0.00000]
[   0.00000]



Замен нет

In [4]:
ch_f = 0
delta = 0.001

while s['x'][2] + s['x'][5] + s['x'][8] + s['x'][11] <= 0:
    ch_f += delta
    
    c = matrix([
        A1_to_B1, # Вывозим из A1 в B1 (а)
        A1_to_B1 + ch_f, # Вывозим из A1 в B1 (б)
        A1_to_B1, # Вывозим из A1 в B1 (а конверченную в б)
        A1_to_B2, # Вывозим из A1 в B2 (а)
        A1_to_B2 + ch_f, # Вывозим из A1 в B2 (б)
        A1_to_B2, # Вывозим из A1 в B2 (а конверченную в б)
        A2_to_B1, # Вывозим из A2 в B1 (а)
        A2_to_B1 + ch_f, # Вывозим из A2 в B1 (б)
        A2_to_B1, # Вывозим из A2 в B1 (а конверченную в б)
        A2_to_B2, # Вывозим из A2 в B2 (а)
        A2_to_B2 + ch_f, # Вывозим из A2 в B2 (б)
        A2_to_B2  # Вывозим из A2 в B2 (а конверченную в б)
    ])
    
    s = solvers.lp(c, G, h, A, b, solver = 'glpk')

print(s['x'])
print(c)
print(ch_f)

[2400.00000]
[   0.00000]
[9000.00000]
[2000.00000]
[3000.00000]
[   0.00000]
[   0.00000]
[   0.00000]
[   0.00000]
[   0.00000]
[   0.00000]
[   0.00000]

[3.40000]
[4.85800]
[3.40000]
[4.30000]
[5.75800]
[4.30000]
[4.80000]
[6.25800]
[4.80000]
[4.30000]
[5.75800]
[4.30000]

1.4579999999999502


Получили 9000